In [1]:
import time

import torch

from childnet import ChildNetwork
from cifar10_training import Cifar10Trainer
from controller import ControllerNetwork
from controller_training import ControllerTrainer

In [2]:
outer_iterations = 310  #310
controller_iterations = 2000  #2000
child_iterations = 400  #400
##
hidden_size = 100
num_layer_types = 4
num_layers = 5
num_filters = 32
num_classes = 10  # cifar-10 dataset
batch_size = 128
# grad_clip_value = 0.15
##
val_accuracy_list = []

train_time_list = []

In [3]:
cifar10_trainer = Cifar10Trainer(batch_size=batch_size)
controller = ControllerNetwork(hidden_size, num_layers, num_layer_types)
controller_trainer = ControllerTrainer(controller, cifar10_trainer, num_hidden_layers=num_layers,
                                       num_filters=num_filters,
                                       num_classes=num_classes, batch_size=batch_size)
load_previous_network_weights = None

what is the device? cpu or gpu ?  cuda
Files already downloaded and verified
Files already downloaded and verified


In [4]:
for i in range(outer_iterations):
    start = time.time()
    print("ITERATION NUMBER %d" % (i + 1))
    ############## sample model ##############
    print('Controller is sampling architecture...')
    soft_1, soft_2, layers_types, skip_connections = controller()
    #
    print(torch.exp(soft_1))
    print(layers_types)
    # print(torch.exp(soft_2))
    # print(skip_connections)
    ############## initialize weights ##############
    print('Creating new child network...')
    if i == 0:
        load_previous_network_weights = False
    child_net = ChildNetwork(layers_types, skip_connections, load_previous_network_weights,
                             num_hidden_layers=num_layers,
                             num_filters=num_filters, num_classes=num_classes)
    load_previous_network_weights = True

    ############## train model ##############
    print('Training child network...')
    # cifar10_trainer.shuffle_and_load_data_again()
    val_accuracy, train_time = cifar10_trainer.train(child_net, num_of_steps=child_iterations)
    val_accuracy_list.append(val_accuracy)
    train_time_list.append(train_time)
    ############## train controller ##############
    print('Training controller network')
    # baseline = weighted_moving_average(val_accuracy_list)
    # multiplier = val_accuracy - baseline  # (reward - baseline)
    # print('val_accuracy is = ', val_accuracy)
    # print('baseline is = ', baseline)
    # print('multiplier is = ', multiplier)
    # controller_trainer.train(num_of_steps=controller_iterations, grad_clip_value=grad_clip_value)
    controller_trainer.train(num_of_steps=controller_iterations)
    ##############################################
    end = time.time()
    total_time = end - start
    print('total time of 1 outer iteration is {:.2f}'.format(total_time))
    print('##############################################')
    print('##############################################')

ITERATION NUMBER 1
Controller is sampling architecture...
tensor([[[0.2610, 0.2406, 0.2439, 0.2544]],

        [[0.2590, 0.2405, 0.2454, 0.2552]],

        [[0.2615, 0.2386, 0.2447, 0.2553]],

        [[0.2597, 0.2409, 0.2449, 0.2544]],

        [[0.2605, 0.2404, 0.2444, 0.2547]],

        [[0.2610, 0.2410, 0.2451, 0.2529]],

        [[0.2610, 0.2409, 0.2428, 0.2553]],

        [[0.2600, 0.2420, 0.2448, 0.2532]],

        [[0.2610, 0.2388, 0.2449, 0.2552]],

        [[0.2609, 0.2412, 0.2419, 0.2559]]], grad_fn=<ExpBackward>)
tensor([[[0., 0., 0., 1.]],

        [[0., 0., 0., 0.]],

        [[0., 0., 0., 1.]],

        [[0., 0., 0., 0.]],

        [[1., 0., 0., 0.]],

        [[0., 0., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 0.]],

        [[0., 1., 0., 0.]],

        [[0., 0., 0., 0.]]])
Creating new child network...
Training child network...
The training time for 400 steps is: 33.91 seconds
The validation time on whole validation set is 1.22 seconds
The validatio

Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(val_accuracy_list)
plt.figure()
plt.plot(train_time_list)

In [ ]:
soft_1, soft_2, layers_types, skip_connections = controller()
print(skip_connections)
print(torch.exp(soft_2))